# Internal Datastructure: Bus branch model, Admittance and Jacobian Matrix


This jupyter notebooks explains how to access and interpret the internal datastructure with relevant matrices.

### Internal Datastructure

We use the simple example network from the create_simple tutorial as an example for how to access internal calculation parameters:

<img src="pics/example_network_simple.png">


In [14]:
import pandapower as pp
import pandapower.networks as nw

net = nw.example_simple()
print(net)

This pandapower network includes the following parameter tables:
   - bus (7 elements)
   - load (1 element)
   - sgen (1 element)
   - gen (1 element)
   - switch (8 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (4 elements)
   - trafo (1 element)


First, we run a power flow in this network:

In [15]:
pp.runpp(net)

When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accesed after power flow:

In [16]:
net._ppc

{'baseMVA': 1,
 'version': 2,
 'bus': array([[ 0.00000000e+00,  3.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.02000000e+00,  0.00000000e+00,
          1.10000000e+02,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  9.60000000e-01,
          1.00000000e+00,  1.02082951e+00,  3.24135661e-02,
          1.10000000e+02,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.02456241e+00,  1.80284793e+00,
          2.00000000e+01,  1.00000000e+00,  2.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.00000000e+00,  2.00000000e+00,  0.00000000e+00,


For information on how this datastructure is defined, please refer to the MATPOWER documentation.

**Note:** For linear power flow (DC load flow) 'Ybus' is no longer created, but 'Bbus' as a new 'internal' key.

In [17]:
pp.rundcpp(net)
net._ppc['internal']['Bbus'].A

array([[ 8402.77777778+0.j, -8402.77777778+0.j,     0.        +0.j,
            0.        +0.j,     0.        +0.j,     0.        +0.j],
       [-8402.77777778+0.j,  8611.22995659+0.j,  -208.45217882+0.j,
            0.        +0.j,     0.        +0.j,     0.        +0.j],
       [    0.        +0.j,  -208.45217882+0.j,  3422.7378931 +0.j,
        -1785.71428571+0.j, -1428.57142857+0.j,     0.        +0.j],
       [    0.        +0.j,     0.        +0.j, -1785.71428571+0.j,
         2092.93394777+0.j,     0.        +0.j,  -307.21966206+0.j],
       [    0.        +0.j,     0.        +0.j, -1428.57142857+0.j,
            0.        +0.j,  1428.57142857+0.j,     0.        +0.j],
       [    0.        +0.j,     0.        +0.j,     0.        +0.j,
         -307.21966206+0.j,     0.        +0.j,   307.21966206+0.j]])

## Nodal Point Admittance Matrix

The nodal point admittance matrix is saved in the ppc and can be accessed directly:

In [18]:
pp.runpp(net)
net._ppc["internal"]["Ybus"].todense()

matrix([[ 2983.234714  -7157.02635809j, -2983.234714  +7159.76331361j,
             0.           +0.j        ,     0.           +0.j        ,
             0.           +0.j        ,     0.           +0.j        ],
        [-2983.234714  +7159.76331361j,  2990.35626947-7364.28068082j,
            -7.11455564 +208.20907269j,     0.           +0.j        ,
             0.           +0.j        ,     0.           +0.j        ],
        [    0.           +0.j        ,    -7.11455564 +208.20907269j,
          1608.40491554-1678.15899439j,  -889.60186671 +816.68368091j,
          -711.68149336 +653.34694473j,     0.           +0.j        ],
        [    0.           +0.j        ,     0.           +0.j        ,
          -889.60186671 +816.68368091j,  1027.81021222 -903.21268537j,
             0.           +0.j        ,  -138.20834551  +86.56929539j],
        [    0.           +0.j        ,     0.           +0.j        ,
          -711.68149336 +653.34694473j,     0.           +0.j        ,
  

Note that the nodal point admittance matrix is given in per unit values.

## Jacobian Matrix

The jacobian Matrix J in the last iteration step is also stored in the ppc and can be accessed:

In [19]:
net._ppc["internal"]["J"].todense()

matrix([[ 9.54796437e+02,  0.00000000e+00, -8.62952855e+02,
          0.00000000e+00, -9.18435819e+01,  0.00000000e+00,
         -9.15296717e+02,  0.00000000e+00, -1.42353628e+02],
        [ 0.00000000e+00,  7.67426447e+03, -2.17432759e+02,
          0.00000000e+00,  0.00000000e+00,  3.05264392e+03,
         -1.38258923e+01,  0.00000000e+00,  0.00000000e+00],
        [-8.60737361e+02, -2.17892547e+02,  1.76161072e+03,
         -6.82980812e+02,  0.00000000e+00, -6.95203313e-01,
          1.64791123e+03, -7.30904816e+02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -6.86871047e+02,
          6.86871047e+02,  0.00000000e+00,  0.00000000e+00,
         -7.26450958e+02,  7.28978162e+02,  0.00000000e+00],
        [-9.18403982e+01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  9.18403982e+01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  1.42355564e+02],
        [ 0.00000000e+00, -3.11622898e+03,  1.41654896e+01,
          0.00000000e+00,  0.000000

The jacobian matrix is also given in per unit values.

## Mapping the Buses

The pandapower indices are not equal to the ppc indices for several reasons. Some buses are fused together in case of closed bus-bus switches and auxiliary buses are created for elements like extended wards or three winding transformers. There is however a mapping between pandapower indices and ppc indices that is created during the conversion to keep track of the dependencies that is also stored in the net:

In [20]:
net._pd2ppc_lookups["bus"]

array([0, 1, 1, 2, 2, 3, 4])

To get a ppc index from the pandapower index, simply call the lookup like this:

In [21]:
pandapower_bus_idx = 3
ppc_index = net._pd2ppc_lookups["bus"][pandapower_bus_idx]
print(ppc_index)

2


As you can see, pandapower bus index 3 corresponds to ppc bus index 2. So if we would like to find the diagonal entry of the Ybus matrix for bus 2, we could now access it with that internal index:

In [22]:
Ybus = net._ppc["internal"]["Ybus"]
int_idx = net._pd2ppc_lookups["bus"][ppc_index]
Ybus[int_idx, int_idx]

array(2990.35626947-7364.28068082j)

We can also see that some buses are mapped to the same internal bus, such as bus 1 and bus 2:

In [23]:
print(net._pd2ppc_lookups["bus"][1])
print(net._pd2ppc_lookups["bus"][2])

1
1


That is because buses 1 and 2 are connected by a closed bus-bus switch and are therefore represented internally as the same bus:

In [24]:
net.switch.loc[0]

bus           1
element       2
et            b
type         CB
closed     True
name       None
z_ohm         0
Name: 0, dtype: object